In [1]:
import os
import random
import time
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from hyperparams import Hyperparameters as params
from stable_baselines3.common.atari_wrappers import (
    ClipRewardEnv, EpisodicLifeEnv, FireResetEnv, MaxAndSkipEnv, NoopResetEnv
)
from stable_baselines3.common.buffers import ReplayBuffer

torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(0)

c:\Users\Harish Vasanth\anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:326: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  np.bool8: (False, True),
c:\Users\Harish Vasanth\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:205: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  np.bool8: (False, True),
c:\Users\Harish Vasanth\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def make_env(env_id, seed, idx, capture_video, run_name):
    def thunk():
        env = gym.make(env_id)
        env = gym.wrappers.RecordEpisodeStatistics(env)
        if capture_video and idx == 0:
            env = gym.wrappers.RecordVideo(env, f"videos/{run_name}")
        env = NoopResetEnv(env, noop_max=30)
        env = MaxAndSkipEnv(env, skip=4)
        env = EpisodicLifeEnv(env)
        if "FIRE" in env.unwrapped.get_action_meanings():
            env = FireResetEnv(env)
        env = ClipRewardEnv(env)
        env = gym.wrappers.ResizeObservation(env, (84, 84))
        env = gym.wrappers.GrayScaleObservation(env)
        env = gym.wrappers.FrameStack(env, 4)
        env.seed(seed)
        env.action_space.seed(seed)
        env.observation_space.seed(seed)
        return env
    return thunk

class QNetwork(nn.Module):
    def __init__(self, action_size):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv2d(4, 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(3136, 512),
            nn.ReLU(),
            nn.Linear(512, action_size)
        )

    def forward(self, x):
        return self.network(x.float() / 255.0)  # Normalize pixel values


In [5]:
run_name = f"{params.env_id}__{params.exp_name}__{params.seed}__{int(time.time())}"

random.seed(params.seed)
np.random.seed(params.seed)
torch.manual_seed(params.seed)
torch.backends.cudnn.deterministic = params.torch_deterministic

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Environment setup
envs = gym.vector.SyncVectorEnv([make_env(params.env_id, params.seed, 0, params.capture_video, run_name)])
assert isinstance(envs.single_action_space, gym.spaces.Discrete), "only discrete action space is supported"

q_network = QNetwork(envs.single_action_space.n).to(device)
optimizer = optim.Adam(q_network.parameters(), lr=params.learning_rate)
target_network = QNetwork(envs.single_action_space.n).to(device)
target_network.load_state_dict(q_network.state_dict())

# Replay buffer setup
rb = ReplayBuffer(
    100_000,
    envs.single_observation_space,
    envs.single_action_space,
    device,
    optimize_memory_usage=False,
    handle_timeout_termination=True,
)

def linear_schedule(start_e: float, end_e: float, duration: int, t: int):
    slope = (end_e - start_e) / duration
    return max(slope * t + start_e, end_e)

obs = envs.reset()
for global_step in range(params.total_timesteps):
    epsilon = linear_schedule(params.start_e, params.end_e, params.exploration_fraction * params.total_timesteps, global_step)

    if random.random() < epsilon:
        actions = np.array([envs.single_action_space.sample() for _ in range(envs.num_envs)])
    else:
        with torch.no_grad():
            state_tensor = torch.as_tensor(obs, device=device, dtype=torch.float32)
            q_values = q_network(state_tensor)
        actions = torch.argmax(q_values, dim=1).cpu().numpy()

    next_obs, rewards, dones, infos = envs.step(actions)

    for idx, info in enumerate(infos):
        if "episode" in info.keys():
            print(f"global_step={global_step}, episodic_return={info['episode']['r']}")
            break

    real_next_obs = next_obs.copy()
    for idx, done in enumerate(dones):
        if done:
            real_next_obs[idx] = None  # Assuming terminal observation is handled differently

    rb.add(obs, real_next_obs, actions, rewards, dones, infos)

    obs = next_obs

    # Training
    if global_step > params.learning_starts and global_step % params.train_frequency == 0:
        data = rb.sample(params.batch_size)
        state_batch = torch.tensor(data.observations, dtype=torch.float32, device=device)
        next_state_batch = torch.tensor(data.next_observations, dtype=torch.float32, device=device)
        action_batch = torch.tensor(data.actions, dtype=torch.long, device=device)
        reward_batch = torch.tensor(data.rewards, dtype=torch.float32, device=device)
        done_batch = torch.tensor(data.dones, dtype=torch.float32, device=device)

        with torch.no_grad():
            next_state_values = target_network(next_state_batch).max(1)[0]
            td_target = reward_batch + params.gamma * next_state_values * (1 - done_batch)

        current_q_values = q_network(state_batch).gather(1, action_batch.unsqueeze(1)).squeeze()
        loss = F.mse_loss(current_q_values, td_target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if global_step % params.target_network_frequency == 0:
        target_network.load_state_dict(q_network.state_dict())

if params.save_model:
    model_path = f"runs/{run_name}/{params.exp_name}_model"
    torch.save(q_network.state_dict(), model_path)
    print(f"model saved to {model_path}")

envs.close()


c:\Users\Harish Vasanth\anaconda3\lib\site-packages\stable_baselines3\common\buffers.py:219: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 5.65GB > 3.58GB
  warnings.warn(


TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'